In [1]:
import os, sys
import pandas as pd
from sklearn.manifold import MDS
from sklearn.cluster import KMeans

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)


from  techminer import *

# import_scopus_file()
import_references()

- INFO - 248 raw records found in ./documents.csv.
- INFO - 5123 raw records found in ./raw-references.csv.
- INFO - Creating references file


100%|██████████| 5123/5123 [01:18<00:00, 65.67it/s]


- INFO - References table saved to ./cited_references_table.csv
           citing_id
cited_id            
1937-0000          3
1943-0000          1
1955-0000          1
1956-0000          2
1960-0000          1
- INFO - References table saved to ./references.csv


In [ ]:
import pandas as pd

documents = pd.read_csv("documents.csv")
documents = documents.sort_values("local_citations", ascending=False)
for row in documents.head(20).iterrows():
    print(row[1]["local_citations"], row[1]["document_title"])
    

In [4]:
import pandas as pd

references = pd.read_csv("references.csv")
references = references.sort_values("local_citations", ascending=True)
for row in references.head(20).iterrows():
    print(row[1]["local_citations"], row[1]["document_title"])

0 Cutoff criteria for fit indexes in covariance structure analysis: Conventional criteria versus new alternatives
0 Past, present and future: music economics at the crossroads
0 Personal information in peer-to-peer loan applications: Is less more?
0 Towards a market entry framework for digital payment platforms
0 Guest editors’ introduction: Figures of entanglement: Special issue introduction
0 Enhancing fundraising with refund bonuses
0 A combination of models for financial crisis prediction: Integrating probabilistic neural network with back-propagation based on adaptive boosting
0 Null hypothesis significance testing: A guide to commonly misunderstood concepts and recommendations for good practice
0 Trust, Security and Privacy in Emerging Distributed Systems
0 Understanding of e-service security dimensions and its effect on quality and intention to use
0 Separating apples from oranges: Identifying threats to financial stability originating from FinTech
0 Revolution of securities law

In [5]:
pd.read_csv("references.csv").local_citations.sum()

0